# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 9:22AM,239079,19,8117533,"Emersa Waterbox, LLC",Released
1,Jun 8 2022 9:18AM,239078,102,PBR-060722-02,"Senseonics, Incorporated",Released
2,Jun 8 2022 9:16AM,239077,10,0085747005,ISDIN Corporation,Released
3,Jun 8 2022 9:16AM,239077,10,0085747006,ISDIN Corporation,Released
4,Jun 8 2022 9:16AM,239077,10,0085746543,ISDIN Corporation,Released
5,Jun 8 2022 9:16AM,239077,10,0085745711,ISDIN Corporation,Released
6,Jun 8 2022 9:16AM,239077,10,0085744191,ISDIN Corporation,Released
7,Jun 8 2022 9:16AM,239077,10,0085745347,ISDIN Corporation,Released
8,Jun 8 2022 9:15AM,239076,10,0085745717,ISDIN Corporation,Released
9,Jun 8 2022 9:15AM,239076,10,0085745719,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,239075,Released,12
46,239076,Released,30
47,239077,Released,6
48,239078,Released,1
49,239079,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239075,NaN,NaN,12.0
239076,NaN,NaN,30.0
239077,NaN,NaN,6.0
239078,NaN,NaN,1.0
239079,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,0.0,14.0,3.0
238984,0.0,1.0,1.0
238989,0.0,1.0,0.0
238992,0.0,0.0,1.0
238994,0.0,56.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,0,14,3
238984,0,1,1
238989,0,1,0
238992,0,0,1
238994,0,56,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,0,14,3
1,238984,0,1,1
2,238989,0,1,0
3,238992,0,0,1
4,238994,0,56,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,,14,3
1,238984,,1,1
2,238989,,1,
3,238992,,,1
4,238994,,56,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 9:22AM,239079,19,"Emersa Waterbox, LLC"
1,Jun 8 2022 9:18AM,239078,102,"Senseonics, Incorporated"
2,Jun 8 2022 9:16AM,239077,10,ISDIN Corporation
8,Jun 8 2022 9:15AM,239076,10,ISDIN Corporation
38,Jun 8 2022 9:14AM,239074,10,ISDIN Corporation
78,Jun 8 2022 9:13AM,239075,20,Alumier Labs Inc.
90,Jun 8 2022 9:06AM,239072,10,Emerginnova
93,Jun 8 2022 8:55AM,239071,10,ISDIN Corporation
135,Jun 8 2022 8:49AM,239070,10,ISDIN Corporation
155,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 9:22AM,239079,19,"Emersa Waterbox, LLC",,,1
1,Jun 8 2022 9:18AM,239078,102,"Senseonics, Incorporated",,,1
2,Jun 8 2022 9:16AM,239077,10,ISDIN Corporation,,,6
3,Jun 8 2022 9:15AM,239076,10,ISDIN Corporation,,,30
4,Jun 8 2022 9:14AM,239074,10,ISDIN Corporation,,,40
5,Jun 8 2022 9:13AM,239075,20,Alumier Labs Inc.,,,12
6,Jun 8 2022 9:06AM,239072,10,Emerginnova,,,3
7,Jun 8 2022 8:55AM,239071,10,ISDIN Corporation,,,42
8,Jun 8 2022 8:49AM,239070,10,ISDIN Corporation,,,20
9,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 9:22AM,239079,19,"Emersa Waterbox, LLC",1,,
1,Jun 8 2022 9:18AM,239078,102,"Senseonics, Incorporated",1,,
2,Jun 8 2022 9:16AM,239077,10,ISDIN Corporation,6,,
3,Jun 8 2022 9:15AM,239076,10,ISDIN Corporation,30,,
4,Jun 8 2022 9:14AM,239074,10,ISDIN Corporation,40,,
5,Jun 8 2022 9:13AM,239075,20,Alumier Labs Inc.,12,,
6,Jun 8 2022 9:06AM,239072,10,Emerginnova,3,,
7,Jun 8 2022 8:55AM,239071,10,ISDIN Corporation,42,,
8,Jun 8 2022 8:49AM,239070,10,ISDIN Corporation,20,,
9,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 9:22AM,239079,19,"Emersa Waterbox, LLC",1,,
1,Jun 8 2022 9:18AM,239078,102,"Senseonics, Incorporated",1,,
2,Jun 8 2022 9:16AM,239077,10,ISDIN Corporation,6,,
3,Jun 8 2022 9:15AM,239076,10,ISDIN Corporation,30,,
4,Jun 8 2022 9:14AM,239074,10,ISDIN Corporation,40,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 9:22AM,239079,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Jun 8 2022 9:18AM,239078,102,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jun 8 2022 9:16AM,239077,10,ISDIN Corporation,6.0,NaN,NaN
3,Jun 8 2022 9:15AM,239076,10,ISDIN Corporation,30.0,NaN,NaN
4,Jun 8 2022 9:14AM,239074,10,ISDIN Corporation,40.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 9:22AM,239079,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Jun 8 2022 9:18AM,239078,102,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jun 8 2022 9:16AM,239077,10,ISDIN Corporation,6.0,0.0,0.0
3,Jun 8 2022 9:15AM,239076,10,ISDIN Corporation,30.0,0.0,0.0
4,Jun 8 2022 9:14AM,239074,10,ISDIN Corporation,40.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3107733,227.0,6.0,24.0
102,239078,1.0,0.0,0.0
12,238984,1.0,1.0,0.0
16,239041,0.0,1.0,0.0
18,478026,2.0,0.0,0.0
19,1912349,47.0,2.0,0.0
20,1673042,66.0,78.0,0.0
21,1673244,7.0,0.0,0.0
22,478120,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3107733,227.0,6.0,24.0
1,102,239078,1.0,0.0,0.0
2,12,238984,1.0,1.0,0.0
3,16,239041,0.0,1.0,0.0
4,18,478026,2.0,0.0,0.0
5,19,1912349,47.0,2.0,0.0
6,20,1673042,66.0,78.0,0.0
7,21,1673244,7.0,0.0,0.0
8,22,478120,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,227.0,6.0,24.0
1,102,1.0,0.0,0.0
2,12,1.0,1.0,0.0
3,16,0.0,1.0,0.0
4,18,2.0,0.0,0.0
5,19,47.0,2.0,0.0
6,20,66.0,78.0,0.0
7,21,7.0,0.0,0.0
8,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,227.0
1,102,Released,1.0
2,12,Released,1.0
3,16,Released,0.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,18,19,20,21,22
Status,,,,,,,,,
Completed,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,1.0,1.0,0.0,2.0,78.0,0.0,1.0
Released,227.0,1.0,1.0,0.0,2.0,47.0,66.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,18,19,20,21,22
0,Completed,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,1.0,1.0,0.0,2.0,78.0,0.0,1.0
2,Released,227.0,1.0,1.0,0.0,2.0,47.0,66.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,18,19,20,21,22
0,Completed,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,1.0,1.0,0.0,2.0,78.0,0.0,1.0
2,Released,227.0,1.0,1.0,0.0,2.0,47.0,66.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()